In [ ]:
!pip -q install langchain openai pinecone langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
from google.colab import userdata
pinecone_api = userdata.get('PINECONE_API_KEY')

In [ ]:
from pinecone import ServerlessSpec

In [ ]:
spec = ServerlessSpec(
    cloud  = "aws",
    region = "us-east-1"
)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
# load embedding
embeddings  = HuggingFaceEmbeddings(model_name = "airesearch/wangchanberta-base-att-spm-uncased")
embed_model = embeddings

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

In [ ]:
embed = embed_model.embed_documents("ฝนตกหนักถล่มภาพเหนือ")
len(embed),len(embed[0])

(20, 768)

In [ ]:
import time
from pinecone import Pinecone

In [ ]:
pc = Pinecone(api_key = pinecone_api)
index_name = "us-place-thai-wangchanberta"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
if index_name not in existing_indexes:
    pc.create_index(index_name,dimension = 768, metric = 'dotproduct', spec = spec)
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)
    print(f"Create vector database name {index_name} successfull !")
else:
    print(f"Aready got vector database name {index_name} !!!")

Aready got vector database name us-place-thai-wangchanberta !!!


In [ ]:
index = pc.Index(index_name)
time.sleep(1)
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'dotproduct',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset(
    "Tun555/us-places-thai",
    split = "train"
)
data = dataset.to_pandas()
data["DESTINATION"] = data["DESTINATION_TH"]
data["NAME"] = data["NAME_TH"]
data["DESCRIPTION"] = data["DESCRIPTION_TH"]
data["REVIEWS"] = data["REVIEWS_TH"]
data = data[['RATING', 'PICTURE','URL', 'DESTINATION', 'NAME', 'DESCRIPTION', 'REVIEWS']]
data

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/135 [00:00<?, ? examples/s]

,RATING,PICTURE,URL,DESTINATION,NAME,DESCRIPTION,REVIEWS
0,4.5,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ออสติน,พิพิธภัณฑ์ศิลปะบลานตัน,"{'categories': array(['แกลเลอรีศิลปะ', 'จุดน่า...","[""พิพิธภัณฑ์ศิลปะที่น่าทึ่ง เราได้ไปที่พิพิธภั..."
1,4.5,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ออสติน,โรงละครโลกเดียว,"{'categories': array(['โรงละคร'], dtype=object...","[""สถานที่จัดคอนเสิร์ตที่ดีที่สุดในเท็กซัส นี่ค..."
2,4.0,None,https://www.tripadvisor.com/Attraction_Review-...,ออสติน,ออสตินอะควาเรียม,"{'categories': array(['ตู้ปลา'], dtype=object)...","[""การเยี่ยมชมภายในบ้านที่ดีเหมาะกับครอบครัวขนา..."
3,5.0,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ออสติน,"""Myo"" can be translated to Thai as ""มิโย"" if i...","{'categories': array(['สปา'], dtype=object), '...","[""ไม่มีการนวดที่แย่เลย ฉันได้ไปเยือนสตูดิโอนวด..."
4,4.5,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ออสติน,บาร์ตัน สปริงส์ พูล,"{'categories': array(['แหล่งน้ำ', 'น้ำพุร้อนแล...","[""คุณเคยเห็นซาลามานเดอร์ในสระน้ำไหม?, นี่ไม่ใช..."
...,...,...,...,...,...,...,...
130,4.5,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ซีแอตเทิล,สวนก๊าซ เวิร์กส์,"{'categories': array(['สวนสาธารณะ'], dtype=obj...","[""คุณต้องไปชมพระอาทิตย์ตกที่อุทยานแก๊สเวิร์ค! ..."
131,5.0,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ซีแอตเทิล,วงจรรีไซเคิล,"{'หมวดหมู่': array(['การเช่าอุปกรณ์'], dtype=o...","[""ประสบการณ์การเช่าจักรยานที่ยอดเยี่ยม จักรยาน..."
132,4.0,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ซีแอตเทิล,สวนประติมากรรมโอลิมปิก,"{'หมวดหมู่': array(['สวนสาธารณะ'], dtype=objec...","[""เดินง่ายพร้อมวิวสวยงาม นี่เป็นกิจกรรมที่ง่าย..."
133,5.0,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ซีแอตเทิล,Seattle Escape Games ترجم إلى Thai: เกมหนีในซี...,"{'categories': array(['เกมหลบหนี'], dtype=obje...","[""ยอดเยี่ยมสำหรับกิจกรรมสร้างทีมงาน เราจองเกมห..."


In [ ]:
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import math
import re, unicodedata, time

# data = dataset.to_pandas()
batch_size = 5

def is_missing(x):
    return x is None or (isinstance(x, float) and math.isnan(x))

def convert_to_native_types(obj):
    if obj is None:
        return ""
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, np.integer):
        return int(obj)
    if isinstance(obj, np.floating):
        return "" if np.isnan(obj) else float(obj)
    if isinstance(obj, (list, tuple)):
        return [convert_to_native_types(v) for v in obj]
    if isinstance(obj, dict):
        return {k: convert_to_native_types(v) for k, v in obj.items()}
    return obj

def create_safe_id(destination, name, idx):
    destination = unicodedata.normalize('NFKD', str(destination or "")).encode('ASCII','ignore').decode('ASCII')
    name        = unicodedata.normalize('NFKD', str(name or "")).encode('ASCII','ignore').decode('ASCII')
    destination = re.sub(r'[^a-zA-Z0-9]+','_', destination).strip('_')
    name        = re.sub(r'[^a-zA-Z0-9]+','_', name).strip('_')
    return f"{destination}_{name}_{idx}"

# Store valid records
valid_ids = []
valid_embeds = []
valid_metadata = []
skipped = 0

for i in tqdm(range(0, len(data), batch_size)):
    batch = data.iloc[i:min(len(data), i+batch_size)]
    for idx, row in batch.iterrows():
        try:
            # Handle both dict or str DESCRIPTION
            desc_obj = row.get('DESCRIPTION', "")
            if isinstance(desc_obj, dict):
                desc_text = desc_obj.get('description', "")
            elif isinstance(desc_obj, str):
                desc_text = desc_obj
            else:
                desc_text = ""

            if not desc_text.strip():
                desc_text = "No description available."

            name = row.get('NAME', "") or ""
            dest = row.get('DESTINATION', "") or ""
            rating = row.get('RATING', 0.0)
            reviews = row.get('REVIEWS', [])
            url = row.get('URL', "") or ""

            # Some datasets have Thai list reviews, so force to list
            if not isinstance(reviews, list):
                reviews = [str(reviews)]

            # Text for embedding
            text_to_embed = f"ชื่อสถานที่: {name}\nสถานที่ตั้ง: {dest}\nรายละเอียด: {desc_text}\nรีวิว: {' '.join(reviews)}"

            embed = embed_model.embed_documents([text_to_embed])[0]
            record_id = create_safe_id(dest, name, idx)

            metadata_entry = convert_to_native_types({
                'text': desc_text,
                'name': name,
                'location': dest,
                'rating': 0.0 if is_missing(rating) else float(rating),
                'reviews': reviews,
                'url': url
            })

            valid_ids.append(record_id)
            valid_embeds.append(embed)
            valid_metadata.append(metadata_entry)

        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            skipped += 1

    # Upsert per batch
    if valid_ids:
        sub_batch_size = 50
        for k in range(0, len(valid_ids), sub_batch_size):
            k_end = min(len(valid_ids), k+sub_batch_size)
            try:
                payload = list(zip(
                    valid_ids[k:k_end],
                    valid_embeds[k:k_end],
                    valid_metadata[k:k_end]
                ))
                index.upsert(vectors=payload)
                print(f"✅ Successfully upserted {k_end - k} records")
            except Exception as e:
                print(f"⚠️ Error upserting batch {k}-{k_end}: {e}")
                if k < len(valid_ids):
                    print(f"First ID in batch: {valid_ids[k]}")
        valid_ids.clear()
        valid_embeds.clear()
        valid_metadata.clear()

print(f"✅ Done! Skipped {skipped} rows due to errors.")


  0%|          | 0/27 [00:00<?, ?it/s]

Error processing row 0: index out of range in self
Error processing row 2: index out of range in self
Error processing row 3: index out of range in self
Error processing row 4: index out of range in self
✅ Successfully upserted 1 records
✅ Successfully upserted 5 records
Error processing row 11: index out of range in self
Error processing row 13: index out of range in self
Error processing row 14: index out of range in self
✅ Successfully upserted 2 records
Error processing row 15: index out of range in self
Error processing row 16: index out of range in self
Error processing row 19: index out of range in self
✅ Successfully upserted 2 records
Error processing row 20: index out of range in self
Error processing row 22: index out of range in self
Error processing row 24: index out of range in self
✅ Successfully upserted 2 records
Error processing row 25: index out of range in self
Error processing row 28: index out of range in self
Error processing row 29: index out of range in self
✅ 

In [ ]:
!pip install -U langchain-pinecone

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.4 MB/s eta 0:00:00


In [ ]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# Initialize Pinecone client
pc = Pinecone(api_key=pinecone_api)

# Connect to index (new API uses .index not .Index)
index = pc.Index("us-place-thai-wangchanberta")   # still works in SDK
# or equivalently
# index = pc.index("us-places-ragv99")

# Create vector store wrapper
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embed_model,    # pass full embedding object, not .embed_query
    text_key="text"
)


/usr/local/lib/python3.12/dist-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [ ]:
query = "อยากเที่ยว พิพิธภัณฑ์"
v = vectorstore.similarity_search(query, k = 3)
for i in v:
    print(i,end='\n')

page_content='{'หมวดหมู่': array(['อาคารสถาปัตยกรรม', 'สถานที่ทางศาสนา'], dtype=object), 'คำอธิบาย': None}' metadata={'location': 'ออสติน', 'name': 'อาสนวิหารเซนต์แมรี่', 'rating': 4.5, 'reviews': ['["การรำลึกถึงรากฐานชาวไอริชของเมือง ชุมชนคาทอลิกชาวไอริชที่ก่อตั้งมหาวิหารเซนต์แมรีในออสติน รัฐเท็กซัส มีต้นกำเนิดในทศวรรษ 1850 เมื่อมีการสร้างโบสถ์หินเล็กๆ ที่ชื่อว่าเซนต์แพทริค บนมุมถนน 9 และบราชอส ในปี 1866 วัดได้ตัดสินใจว่าพวกเขาต้องการโบสถ์ใหม่ ในปี 1872 หลังจากที่ออสตินถูกกำหนดให้เป็นเมืองหลวงถาวรของเท็กซัส วัดได้วางศิลาฤกษ์สำหรับโบสถ์ใหม่ที่ 203 ถนนอีสต์ 10 ซึ่งอยู่หนึ่งช่วงตึกทางเหนือจากอาคารเดิม โบสถ์เสร็จสมบูรณ์ในปี 1874 และมีการถวายในมหาวิหารเซนต์แมรีในปี 1884 ออกแบบโดยสถาปนิกเกิดที่ไอร์แลนด์ นิโคลัส เจ. เคลย์ตัน สไตล์ของมันเป็นแบบวิคตอเรียที่มีหอคอยและยอดแบบนีโอ-โกธิก เป็นหนึ่งในระฆังที่ใหญ่ที่สุดในรัฐ มีหน้าต่างกระจกสี โค้งไม้ที่ตั้งอยู่บนประตูและหน้าต่าง ภาพจิตรกรรมสีสันสดใส และเพดานและโดมสีน้ำเงินที่ประดับด้วยดวงดาวซึ่งสะท้อนท้องฟ้าและสวรรค์ มหาวิหาร underwent การบูรณะอย่างกว

In [ ]:
query = "อยากเที่ยวได้ รายละเอียดเกี่ยวกับการท่องเที่ยวแนวธรรมชาติ"
v = vectorstore.similarity_search(query, k = 3)
for i in v:
    print(i,end='\n')

page_content='{'categories': array(['พิพิธภัณฑ์เฉพาะทาง'], dtype=object), 'description': 'ที่ตั้งสำนักงานใหญ่ดั้งเดิมของมิชชั่นนารีโปรเตสแตนท์ชาวอเมริกันในศตวรรษที่ 19 พิพิธภัณฑ์นี้อธิบายประวัติศาสตร์ของมิชชั่นนารีอเมริกันในฮาวาย.'}' metadata={'location': 'โออาฮู', 'name': 'ไซต์ประวัติศาสตร์และสถาบันเก็บเอกสารบ้านมิชชันนารีฮาวาย', 'rating': 4.5, 'reviews': ['["ทัวร์ที่น่าสนใจและน่าดึงดูดใจ ฉันตั้งใจที่จะทำทัวร์นี้มาเป็นเวลานานและได้วางแผนสำหรับมันในการกลับมาที่โออาฮูที่รอคอยมานานของฉัน\\n\\nในกลุ่มของเรามีเพียงสามคนรวมทั้งไกด์ของเรา ไมค์ ที่เต็มไปด้วยข้อมูลทางประวัติศาสตร์ เขาได้ทำให้วันที่แสนเริ่มต้นมีชีวิตชีวาขึ้นด้วยข้อเท็จจริงและเล่าเรื่องราว ควบคู่กับเรา ไมค์พาเราไปอยู่ในภาพยนตร์ "ฮาวาย" ข้างๆ จูลี่ แอนดรูว์ ในชั่วขณะหนึ่ง\\n\\nคุณจะได้เยี่ยมชมสามอาคาร ซึ่งได้แก่ อาคารดั้งเดิมและเก่าแก่ที่สุดในโฮโนลูลู บ้านมิชชันนารีแห่งที่สอง และร้านพิมพ์ที่น่าทึ่งซึ่งเป็นที่ที่ภาษาเฮาชันถูกบันทึกลงบนกระดาษเป็นครั้งแรก\\n\\nร้านของขวัญเต็มไปด้วยงานฝีมือของชาวฮาวายและหนังสือดีๆ มากมาย คุณสามารถนั่

In [ ]:
queries = [
    "อยากไปเที่ยวพิพิธภัณฑ์ในไมอามี่",
    "สถานที่เกี่ยวกับศิลปะและวัฒนธรรม" ,
    "อยากดูภาพจิตรกรรมฝาผนังหรือกราฟฟิตี้",
    "มีที่ไหนที่จัดแสดงผลงานศิลปะบ้าง",
    "อยากได้สถานที่ท่องเที่ยวที่เป็นมรดกทางวัฒนธรรม",
    "อยากเที่ยวกับครอบครัว",
    "สถานที่ที่เหมาะกับการพักผ่อน",
    "อยากไปถ่ายรูปสวยๆ",
    "สถานที่มีกิจกรรมสนุกๆ ให้ทำ",
    "อยากไปที่เหมาะกับเด็กๆ",
    "สถานที่ที่มีทั้งพิพิธภัณฑ์และสวนสวย",
    "อยากไปที่ที่มีสัตว์และสามารถเรียนรู้ได้ด้วย",
    "อยากเที่ยวแบบไม่ใช่ในตัวเมือง",
    "อยากได้ที่เที่ยวที่มีทั้งศิลปะและธรรมชาติ",
    "สถานที่ที่สามารถเรียนรู้ประวัติศาสตร์ของไมอามี่"
]
for q in queries:
    print(q)
    v = vectorstore.similarity_search_with_score(q,k=5)
    for i,score in v:
        page = str(i.page_content)
        if "array([" in page:
            try:
                cat_part = page.split("array([")[1].split("],")[0]
            except Exception:
                cat_part = "-"
        else:
            cat_part = "-"
        # print(i)
        print(f"- Place : {i.metadata.get('name','?')}")
        print(f"    - Score    : {score:.5f}")
        print(f"    - Category : {cat_part}")
        print(f"    - Position : {i.metadata.get('location','?')}")
        print(f"    - url      : {i.metadata.get('url','?')}")

    print()


อยากไปเที่ยวพิพิธภัณฑ์ในไมอามี่
- Place : North End ภาษาไทยคือ "ตอนเหนือ"
    - Score    : 160.59131
    - Category : 'ย่าน'
    - Position : บอสตัน
    - url      : https://www.tripadvisor.com/Attraction_Review-g60745-d103113-Reviews-North_End-Boston_Massachusetts.html
- Place : เสรีภาพทาวเวอร์ที่วิทยาลัยไมอามีเดด
    - Score    : 157.13037
    - Category : 'สถานที่ประวัติศาสตร์', 'อาคารสถาปัตยกรรม'
    - Position : ไมอามี
    - url      : https://www.tripadvisor.com/Attraction_Review-g34438-d532100-Reviews-Freedom_Tower_at_Miami_Dade_College-Miami_Florida.html
- Place : สวนพฤกษศาสตร์ปล่องภูเขาไฟโคโค่
    - Score    : 154.10645
    - Category : 'สวน'
    - Position : โอะอาฮู
    - url      : https://www.tripadvisor.com/Attraction_Review-g60982-d560888-Reviews-Koko_Crater_Botanical_Garden-Honolulu_Oahu_Hawaii.html
- Place : พีระมิดทรานซาเมอริกา
    - Score    : 153.61426
    - Category : 'สถานที่น่าสนใจและสถานที่สำคัญ', 'อาคารสถาปัตยกรรม'
    - Position : ซานฟรานซิสโก
    - url      : 